In [1]:
import load 
import generation
import pandas as pd
import matplotlib.pyplot as plt

import importlib
import main


# Na het aanpassen van mijn_module.py:
importlib.reload(main)
importlib.reload(load)
importlib.reload(generation)

<module 'generation' from '/home/rayen/Documents/python/energy/energy/generation.py'>

In [31]:

m_start = '06'  # Example: December
d_start = '01'  # Start day
y_start = '2024'


from datetime import datetime

now = datetime.now()



y_end = str(now.year)
m_end = str(now.month).zfill(2)
d_end = str(now.day).zfill(2)

print(y_end, m_end, d_end)



# d_end = '03'    # End day
# m_end = '01'    # End month
# y_end = '2025'

tables = main.load_tables(m_start, d_start, m_end, d_end, y_start, y_end)

2025 04 22
Data A65_A01 Retrieved Successfully
Data A65_A16 Retrieved Successfully
Data A75_A16 Retrieved Successfully


Fetching Data Progress: 100%|██████████| 163/163 [11:06<00:00,  4.09s/segment] 


In [32]:
tables['weather_data']

,timestamp,STN,DD,FH,FF,FX,T,T10N,TD,SQ,...,VV,N,U,WW,IX,M,R,S,O,Y
0,2024-06-01 00:00:00+00:00,260,290,20,20,50,143,NaN,138,0,...,65,8.0,97,51.0,7,0,1,0,0,0
1,2024-06-01 01:00:00+00:00,260,10,20,20,40,150,NaN,147,0,...,56,8.0,98,81.0,7,0,1,0,0,0
2,2024-06-01 02:00:00+00:00,260,360,40,40,80,153,NaN,149,0,...,60,8.0,97,23.0,7,0,1,0,0,0
3,2024-06-01 03:00:00+00:00,260,360,30,30,70,153,NaN,149,0,...,58,8.0,97,10.0,7,0,0,0,0,0
4,2024-06-01 04:00:00+00:00,260,340,40,40,90,153,NaN,150,0,...,57,8.0,98,23.0,7,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7771,2025-04-20 19:00:00+00:00,260,20,30,30,60,113,NaN,73,0,...,83,8.0,76,NaN,5,0,0,0,0,0
7772,2025-04-20 20:00:00+00:00,260,40,20,20,50,109,NaN,75,0,...,83,8.0,79,NaN,5,0,0,0,0,0
7773,2025-04-20 21:00:00+00:00,260,70,10,10,30,108,NaN,78,0,...,83,8.0,81,NaN,5,0,0,0,0,0
7774,2025-04-20 22:00:00+00:00,260,60,10,10,20,100,NaN,86,0,...,60,8.0,91,81.0,7,0,1,0,0,0


In [ ]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
from sqlalchemy import create_engine


def list_tables():
    # Database credentials (same as your existing setup)
    DB_NAME = "energy"
    DB_USER = "rayen"
    DB_PASSWORD = "xxx"
    DB_HOST = "localhost"
    DB_PORT = "5432"

    # Create a database connection
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    
    # Query to list all tables in the public schema
    query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public' 
        AND table_type = 'BASE TABLE';
    """
    
    # Load results into a DataFrame
    df = pd.read_sql(query, engine)
    
    # Return tables as a list
    return df['table_name'].tolist()


def insert_dataframe_with_unique_constraint(df, table_name, unique_columns, db_config, if_exists="replace"):
 

    # Standardize columns
    df = df.copy()  # avoid SettingWithCopyWarning
    df.columns = df.columns.str.lower().str.replace(" ", "_")

    # Convert unique_columns to list if it's a string
    if isinstance(unique_columns, str):
        unique_columns = [unique_columns]

    # Convert timestamp to datetime if present in unique columns
    for col in unique_columns:
        if col in df.columns and col == "timestamp":
            df[col] = pd.to_datetime(df[col], errors="coerce")

    # Create the engine
    engine = create_engine(
        f"postgresql://{db_config['user']}:{db_config['password']}@"
        f"{db_config['host']}:{db_config['port']}/{db_config['dbname']}"
    )

    # Define a schema (e.g., 'public')
    schema_name = "public"

    # Insert the DataFrame into PostgreSQL
    df.to_sql(table_name, engine, if_exists=if_exists, index=False, schema=schema_name)

    # Build the constraint name
    constraint_name = f"{table_name}_" + "_".join(unique_columns) + "_unique"
    unique_columns_str = ", ".join(unique_columns)

    # Construct the ALTER TABLE query (with schema + quoted table name)
    alter_table_query = f"""
        ALTER TABLE "{schema_name}"."{table_name}"
        ADD CONSTRAINT "{constraint_name}" UNIQUE ({unique_columns_str});
    """

    try:
        conn = psycopg2.connect(
            dbname=db_config["dbname"],
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=db_config["port"]
        )
        cur = conn.cursor()
        cur.execute(alter_table_query)
        conn.commit()
        print(f"✅ Unique constraint added to {unique_columns_str} in {schema_name}.{table_name}!")
    except Exception as e:
        print("❌ Error while adding unique constraint:", e)
    finally:
        cur.close()
        conn.close()

    print(f"✅ DataFrame successfully inserted into '{schema_name}.{table_name}' table!")


def upsert_dataframe(df, table_name, conflict_columns, db_config, schema_name="public"):
    """
    Upserts data from a DataFrame into a PostgreSQL table in the specified schema.
    
    Parameters:
      df (pd.DataFrame): The DataFrame to upsert.
      table_name (str): The name of the target table.
      conflict_columns (str or list): Column(s) to check for conflict (e.g., primary key).
      db_config (dict): Database connection parameters with keys: dbname, user, password, host, port.
      schema_name (str): The PostgreSQL schema where the table resides. Defaults to "public".
    """
    # Make a copy to avoid SettingWithCopyWarning if we're going to mutate columns
    df = df.copy()
    
    # Convert 'timestamp' to datetime if present
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    
    # Standardize column names: lowercase and replace spaces with underscores
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    
    # Convert conflict_columns to list if it is a string
    if isinstance(conflict_columns, str):
        conflict_columns = [conflict_columns]
    
    # Convert DataFrame to list of tuples for SQL execution
    records = df.to_records(index=False).tolist()
    
    # Get column names
    columns = list(df.columns)
    columns_str = ", ".join(columns)
    
    # Build the update clause (exclude conflict columns)
    update_cols = [col for col in columns if col not in conflict_columns]
    update_clause = ", ".join([f"{col} = EXCLUDED.{col}" for col in update_cols])
    
    # Build the conflict target string
    conflict_str = ", ".join(conflict_columns)
    
    # Construct the UPSERT query with schema + quoted table name
    upsert_query = f"""
    INSERT INTO "{schema_name}"."{table_name}" ({columns_str})
    VALUES %s
    ON CONFLICT ({conflict_str}) DO UPDATE
    SET {update_clause};
    """
    
    # Connect to the database
    try:
        conn = psycopg2.connect(
            dbname=db_config["dbname"],
            user=db_config["user"],
            password=db_config["password"],
            host=db_config["host"],
            port=db_config["port"]
        )
        cur = conn.cursor()
        
        # Execute batch upsert
        execute_values(cur, upsert_query, records)
        
        conn.commit()
        print("✅ Data upserted successfully!")
        
    except Exception as e:
        print("❌ Error:", e)
    finally:
        cur.close()
        conn.close()


def load_table(table):

    # Database credentials
    DB_NAME = "energy"
    DB_USER = "rayen"
    DB_PASSWORD = "xxx"
    DB_HOST = "localhost"
    DB_PORT = "5432"


    # Create a database connection using SQLAlchemy
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

    # SQL query to load the table
    query = f"SELECT * FROM {table};"

    # Load data into a Pandas DataFrame
    df = pd.read_sql(query, engine)

    return df


def delete_table(table):

    # Database credentials
    DB_NAME = "energy"
    DB_USER = "rayen"
    DB_PASSWORD = "xxx"
    DB_HOST = "localhost"
    DB_PORT = "5432"

    try:
        # Connect to the database
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        cur = conn.cursor()

        print('deleting ',table)

        # SQL query to drop the table
        drop_table_query = f"DROP TABLE IF EXISTS {table} CASCADE;"
        
        # Execute the query
        cur.execute(drop_table_query)
        
        # Commit changes
        conn.commit()
        
        print(f"✅ Table '{table}' dropped successfully!")

        # Close the connection
        cur.close()
        conn.close()

    except Exception as e:
        print("❌ Error:", e)


def reset_database():
    
    DB_CONFIG = {
        "dbname": "energy",  # Connect to the default system DB
        "user": "rayen",
        "password": "xxx",
        "host": "localhost",
        "port": "5432"
    }

    try:
        conn = psycopg2.connect(**DB_CONFIG)
        conn.autocommit = True
        cur = conn.cursor()

        # Force disconnect all users before dropping
        cur.execute("""
            SELECT pg_terminate_backend(pg_stat_activity.pid)
            FROM pg_stat_activity
            WHERE pg_stat_activity.datname = 'mydatabase' AND pid <> pg_backend_pid();
        """)

        cur.execute("DROP DATABASE IF EXISTS mydatabase;")
        cur.execute("CREATE DATABASE mydatabase;")

        print("Database reset successfully.")

        cur.close()
        conn.close()

    except Exception as e:
        print("Error:", e)

In [33]:
list_tables()



['weather_data', 'A01_data', 'A16_data', 'generation_data', 'market_data']

In [ ]:
# Database configuration dictionary
db_config = {
"dbname": "energy",
"user": "rayen",
"password": "xxx",
"host": "localhost",
"port": "5432"
}

# Load the DataFrame (for example, the first 200 rows)

for table in tables.keys():

    df = tables[table]

    insert_dataframe_with_unique_constraint(df, table, 'timestamp', db_config, if_exists="replace")




✅ Unique constraint added to timestamp in public.A01_data!
✅ DataFrame successfully inserted into 'public.A01_data' table!
✅ Unique constraint added to timestamp in public.A16_data!
✅ DataFrame successfully inserted into 'public.A16_data' table!
✅ Unique constraint added to timestamp in public.generation_data!
✅ DataFrame successfully inserted into 'public.generation_data' table!
✅ Unique constraint added to timestamp in public.market_data!
✅ DataFrame successfully inserted into 'public.market_data' table!
✅ Unique constraint added to timestamp in public.weather_data!
✅ DataFrame successfully inserted into 'public.weather_data' table!


In [ ]:
# Database configuration dictionary
db_config = {
"dbname": "energy",
"user": "rayen",
"password": "xxx",
"host": "localhost",
"port": "5432"
}

for table_name, df_data in tables.items():

    print(table_name)


    upsert_dataframe(
        df_data,
        table_name=table_name,        # e.g. "market_data"
        conflict_columns="timestamp",
        db_config=db_config,
        schema_name="public"
    )


A01_data
✅ Data upserted successfully!
A16_data
✅ Data upserted successfully!
generation_data
✅ Data upserted successfully!
market_data
✅ Data upserted successfully!
weather_data
❌ Error: bigint out of range



In [ ]:
for table in tables.keys():

    table_name = f'public.{table}'

    delete_table(table_name)


✅ Table 'table' dropped successfully!
✅ Table 'table' dropped successfully!
✅ Table 'table' dropped successfully!
✅ Table 'table' dropped successfully!
✅ Table 'table' dropped successfully!


In [35]:
load_table('public.market_data')

,timestamp,index,position,price
0,2024-06-01 00:00:00+02:00,0,1,67.86
1,2024-06-01 01:00:00+02:00,1,2,67.77
2,2024-06-01 02:00:00+02:00,2,3,70.30
3,2024-06-01 03:00:00+02:00,3,4,57.93
4,2024-06-01 04:00:00+02:00,4,5,63.27
...,...,...,...,...
7784,2025-04-22 19:00:00+02:00,19,20,178.72
7785,2025-04-22 20:00:00+02:00,20,21,253.35
7786,2025-04-22 21:00:00+02:00,21,22,157.73
7787,2025-04-22 22:00:00+02:00,22,23,121.00


In [195]:
tables = list_tables()

print(tables)



['weather_data', 'A01_data', 'A16_data', 'generation_data', 'market_data']


In [181]:
for table in tables:

    table_name = f'{table}'

    delete_table(table_name)

deleting  A01_data
✅ Table 'A01_data' dropped successfully!
deleting  A16_data
✅ Table 'A16_data' dropped successfully!
